In [20]:
import numpy as np
import math

class Stp_MADS:
    def __init__(self, x0, bounds, delta0=1.0, delta_min=1e-6, alpha=2.0, gamma=0.5):
        self.x = np.array(x0)
        self.bounds = np.array(bounds)
        self.delta = delta0
        self.delta_min = delta_min
        self.alpha = alpha
        self.gamma = gamma
        self.mesh_size = delta0
        self.iteration = 0
        self.num_call = 0
        self.pending_eval = []
        self.iter_eval = None
        self.best = math.inf
        self.evaluations = []
    
    def get_next_candidates(self):
        if len(self.pending_eval)!=0:
            raise RuntimeError("Previous evaluation result is still pending.")
        
        directions = np.eye(len(self.x))
        candidates = [self.x + self.mesh_size * d for d in directions] + [self.x - self.mesh_size * d for d in directions]
        candidates = [np.clip(c, self.bounds[:, 0], self.bounds[:, 1]) for c in candidates]
        self.pending_eval = candidates
        self.iter_eval = candidates.copy()
        
        return candidates
    
    def stp_update_with_result(self, evaluation):
        if self.pending_eval == []:
            self.update_with_result()
        else:        
            self.num_call += 1
            self.evaluations.append(evaluation)

        return self.pending_eval.pop(0)
    
    def update_with_result(self):

        best_idx = np.argmin(self.evaluations)
        best_candidate = self.iter_eval[best_idx]
        best_value = self.evaluations[best_idx]


        if best_value < self.best:
            self.x = best_candidate
            self.mesh_size *= self.alpha
            self.best = best_value
        else:
            self.mesh_size *= self.gamma
        
        self.mesh_size = max(self.mesh_size, self.delta_min)
        self.pending_eval = []
        self.get_next_candidates()
        self.iteration += 1
        self.evaluations = []
 
    
    
    def get_current_solution(self):
        return self.x

# Example Usage
bounds = np.array([[0, 1], [10, 20], [0, 3]])
mads = Stp_MADS(x0=[0.9, 19, 0.1], bounds=bounds)
def obj_func(x_next):
    return (x_next[0] - 0.5)**2 + (x_next[1] - 15)**2 + (x_next[2] - 1.5)**2


mads.get_next_candidates()
candidate = mads.pending_eval[0]
print("mads.iter_eval", mads.iter_eval)
for _ in range(200):  # Run for a few iterations

    evaluation = obj_func(candidate)
    candidate = mads.stp_update_with_result(evaluation)
    print("candidate, evaluation", candidate, evaluation)
    print(f"Iteration {mads.iteration} {mads.num_call}: Best solution {mads.get_current_solution()}, {float(mads.best)}, {mads.mesh_size}")
    print(print("mads.iter_eval", _, mads.iter_eval))

mads.iter_eval [array([ 1. , 19. ,  0.1]), array([ 0.9, 20. ,  0.1]), array([ 0.9, 19. ,  1.1]), array([ 0. , 19. ,  0.1]), array([ 0.9, 18. ,  0.1]), array([ 0.9, 19. ,  0. ])]
candidate, evaluation [ 1.  19.   0.1] 18.21
Iteration 0 1: Best solution [ 0.9 19.   0.1], inf, 1.0
mads.iter_eval 0 [array([ 1. , 19. ,  0.1]), array([ 0.9, 20. ,  0.1]), array([ 0.9, 19. ,  1.1]), array([ 0. , 19. ,  0.1]), array([ 0.9, 18. ,  0.1]), array([ 0.9, 19. ,  0. ])]
None
candidate, evaluation [ 0.9 20.   0.1] 18.21
Iteration 0 2: Best solution [ 0.9 19.   0.1], inf, 1.0
mads.iter_eval 1 [array([ 1. , 19. ,  0.1]), array([ 0.9, 20. ,  0.1]), array([ 0.9, 19. ,  1.1]), array([ 0. , 19. ,  0.1]), array([ 0.9, 18. ,  0.1]), array([ 0.9, 19. ,  0. ])]
None
candidate, evaluation [ 0.9 19.   1.1] 27.12
Iteration 0 3: Best solution [ 0.9 19.   0.1], inf, 1.0
mads.iter_eval 2 [array([ 1. , 19. ,  0.1]), array([ 0.9, 20. ,  0.1]), array([ 0.9, 19. ,  1.1]), array([ 0. , 19. ,  0.1]), array([ 0.9, 18. ,  0.1

In [29]:
import numpy as np

class StepwiseMADS:
    def __init__(self, x0, bounds, delta0=1.0, delta_min=1e-3, alpha=2.0, gamma=0.5):
        self.x = np.array(x0)
        self.bounds = np.array(bounds)
        self.delta = delta0
        self.delta_min = delta_min
        self.alpha = alpha
        self.gamma = gamma
        self.mesh_size = delta0
        self.iteration = 0
        self.pending_eval = None
        self.best_value = None  # Store the best evaluation value
    
    def get_next_candidate(self):
        if self.pending_eval is not None:
            raise RuntimeError("Previous evaluation result is still pending.")
        
        directions = np.eye(len(self.x))
        candidates = [self.x + self.mesh_size * d for d in directions] + [self.x - self.mesh_size * d for d in directions]
        
        candidates = [np.clip(c, self.bounds[:, 0], self.bounds[:, 1]) for c in candidates]
        self.pending_eval = candidates
        return candidates
    
    def update_with_result(self, evaluations):
        if self.pending_eval is None:
            raise RuntimeError("No pending evaluations to update with.")
        
        best_idx = np.argmin(evaluations)
        best_candidate = self.pending_eval[best_idx]
        best_value = evaluations[best_idx]
        
        if self.best_value is None or best_value < self.best_value:
            self.x = best_candidate
            self.best_value = best_value
            self.mesh_size *= self.alpha
        else:
            self.mesh_size *= self.gamma
        
        self.mesh_size = max(self.mesh_size, self.delta_min)
        self.pending_eval = None
        self.iteration += 1
    
    def get_current_solution(self):
        return self.x

# Example Usage
def evaluate(x):
    a, b, c = x
    return (a - 0.5)**2 + (b - 15)**2 + (c - 1.5)**2  # Example function

bounds = np.array([[0, 1], [10, 20], [0, 3]])
mads = StepwiseMADS(x0=[0.9, 19, 0.1], bounds=bounds)
mads.best_value = evaluate(mads.x)  # Initialize best value with initial point evaluation

for _ in range(20):  # Run for a few iterations
    candidates = mads.get_next_candidate()
    evaluations = [evaluate(c) for c in candidates]  # Simulated evaluation
    mads.update_with_result(evaluations)
    print(f"Iteration {_}: Best solution {mads.get_current_solution()} with value {mads.best_value}")


Iteration 0: Best solution [ 0.9 18.   0.1] with value 11.12
Iteration 1: Best solution [ 0.9 16.   0.1] with value 3.12
Iteration 2: Best solution [ 0.9 16.   0.1] with value 3.12
Iteration 3: Best solution [ 0.9 16.   2.1] with value 1.5200000000000002
Iteration 4: Best solution [ 0.9 16.   2.1] with value 1.5200000000000002
Iteration 5: Best solution [ 0.9 16.   2.1] with value 1.5200000000000002
Iteration 6: Best solution [ 0.9 15.   2.1] with value 0.5200000000000001
Iteration 7: Best solution [ 0.9 15.   2.1] with value 0.5200000000000001
Iteration 8: Best solution [ 0.9 15.   1.1] with value 0.31999999999999995
Iteration 9: Best solution [ 0.9 15.   1.1] with value 0.31999999999999995
Iteration 10: Best solution [ 0.9 15.   1.1] with value 0.31999999999999995
Iteration 11: Best solution [ 0.4 15.   1.1] with value 0.16999999999999993
Iteration 12: Best solution [ 0.4 15.   1.1] with value 0.16999999999999993
Iteration 13: Best solution [ 0.4 15.   1.6] with value 0.0200000000000